In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/卒業制作/QAIZ/'

In [ ]:
!pip install -r ./examples/pytorch/summarization/requirements.txt
!pip install git+https://github.com/huggingface/transformers
!pip install datasets
!pip install wikipedia
!pip install transformers[ja]

In [ ]:
from gensim import models
w2v_model =  models.KeyedVectors.load_word2vec_format('jawiki.word_vectors.200d.txt', binary=False)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import wikipedia
import random

wikipedia.set_lang("ja")

# モデルのロード
model = AutoModelForSeq2SeqLM.from_pretrained('summary_ja/')
tokenizer = AutoTokenizer.from_pretrained('sonoisa/t5-base-japanese')

# Wikipediaからテキストを取得
word = "織田信長"
text = wikipedia.page(word).content

# テキストをトークン化してモデルに入力
inputs = tokenizer(text, return_tensors="pt", max_length=1000, truncation=True)

# モデルからの生成
outputs = model.generate(inputs["input_ids"], max_length=100, min_length=20,return_dict_in_generate=True, output_scores=True,
    temperature=1.0,
    num_beams=10,
    diversity_penalty=1.0,
    num_beam_groups=10,
    num_return_sequences=10,
    repetition_penalty=1.5)


output_text = [tokenizer.decode(ids, skip_special_tokens=True,
                                     clean_up_tokenization_spaces=False)
                    for ids in outputs.sequences]

for i, title in enumerate(output_text):
  try:
    question, answer = output_text[i].split("?")
    results = w2v_model.most_similar(positive=[answer])
    incorrect_answer1, incorrect_answer2, incorrect_answer3 = results[0][0], results[1][0], results[2][0]
    print(f'{i+1}:')
    print(f'入力: {word}')
    print(f'問題: {question}?')
    print(f'答えの選択肢: {answer}')
    print(f'間違いの選択肢: {incorrect_answer1}, {incorrect_answer2}, {incorrect_answer3}\n')
  except:
    print(f'{i+1}:')
    print(f"create question error:{output_text[i]}\n")
  pass


1:
入力: 織田信長
問題: 弾正忠家の当主・織田信秀の嫡男で、織田信長の後を継いだのは誰?
答えの選択肢: 織田信秀
間違いの選択肢: 松井忠次, 池田輝政, 斎藤道三

2:
create question error:、織田信長の「三英傑」と呼ばれる人物は誰?織田信長 織田信長の「三英傑」は誰?織田信長

3:
入力: 織田信長
問題: 、織田信長が天下人になったのは、桶狭間の戦いの最中だったと言われ、桶狭間の戦いで今川義元を討ち取ったことで有名です。また、桶狭間の戦いでは、今川義元を討ち取ったことで有名な、三英傑の一人は何?
答えの選択肢: 織田信長
間違いの選択肢: 徳川家康, 豊臣秀吉, 今川義元

4:
create question error:織田信長は、尾張一国の支配を確立したことで知られる、安土桃山時代の大名は何?織田信秀 織田信長は、戦国時代から安土桃山時代にかけて活躍した、日本の戦国時代を代表する英雄は何?織田信長

5:
入力: 織田信長
問題: 弾正忠家、織田弾正忠家、織田弾正忠家の3家からなる「織田政権」を築いた、日本の戦国時代を代表する英雄は何?
答えの選択肢: 織田信長
間違いの選択肢: 徳川家康, 豊臣秀吉, 今川義元

6:
入力: 織田信長
問題: の戦国大名・織田信秀の嫡男で、織田信長の後を継いだのは誰?
答えの選択肢: 織田信秀
間違いの選択肢: 松井忠次, 池田輝政, 斎藤道三

7:
入力: 織田信長
問題: の戦国大名・織田信長の嫡男で、その生涯を「三英傑」と総称される人物は誰?
答えの選択肢: 織田信長
間違いの選択肢: 徳川家康, 豊臣秀吉, 今川義元

8:
入力: 織田信長
問題: 、徳川家康や徳川家康を滅ぼし、天下人となった戦国の三英傑の一人は誰?
答えの選択肢: 織田信長
間違いの選択肢: 徳川家康, 豊臣秀吉, 今川義元

9:
入力: 織田信長
問題: で、織田信長は「三英傑」と呼ばれる、織田信長が残した功績を何という?
答えの選択肢: 織田信長
間違いの選択肢: 徳川家康, 豊臣秀吉, 今川義元

10:
入力: 織田信長
問題: に「信長」というあだ名をつけられたことがある、織田信長は、どのような人物だった?
答えの選択肢: 織田信長
間違いの選択肢: 徳川家康, 豊臣秀吉, 今川義元

